# Import

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn 

from glob import glob

# vis 
import matplotlib.pyplot as plt
import matplotlib 
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from category_encoders import OrdinalEncoder

# modeling
import pandas as pd 
import numpy as np 

from sklearn.model_selection import KFold
from tqdm import tnrange, tqdm_notebook

import optuna
from optuna.samplers import TPESampler

import lightgbm as lgb
from lightgbm import LGBMRegressor, LGBMClassifier

# 성능 평가
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score , roc_curve
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix

# 
from sklearn.cluster import KMeans

#
import joblib

# 
from sklearn.cluster import DBSCAN
import lightgbm as lgb
from imblearn.over_sampling import *

# 
# MinMaxScaling
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Load dataset

In [19]:
X_smote_ = pd.read_csv("..\\..\\data\\X_smote.csv")
y_smote_ = pd.read_csv("..\\..\\data\\y_smote.csv")
# X_train = pd.read_csv("..\\..\\data\\X_train.csv")
# y_train = pd.read_csv("..\\..\\data\\y_train.csv")
X_val_ = pd.read_csv("..\\..\\data\\X_val.csv")
y_val_ = pd.read_csv("..\\..\\data\\y_val.csv")
X_test_ = pd.read_csv("..\\..\\data\\X_test.csv")
y_test_ = pd.read_csv("..\\..\\data\\y_test.csv")

In [20]:
X_smote = X_smote_.copy()
y_smote = y_smote_.copy()
X_val = X_val_.copy()
y_val = y_val_.copy()
X_test = X_test_.copy()
y_test = y_test_.copy()

In [21]:
best_params = {'boosting_type': 'dart', 'learning_rate': 0.44030778031031004, 'n_estimators': 530, 'max_depth': 28, 'num_leaves': 162, 'reg_alpha': 0.38178611895252984, 'reg_lambda': 0.5536690972721715, 'subsample': 0.5595879976748785, 'subsample_freq': 30, 'colsample_bytree': 0.4086346398917182, 'min_child_samples': 42, 'max_bin': 94}

In [22]:
X_test.columns

Index(['loan_limit', 'loan_rate', 'gender', 'credit_score', 'yearly_income',
       'income_type', 'employment_type', 'houseown_type', 'desired_amount',
       'purpose', 'personal_rehabilitation_yn', 'existing_loan_cnt',
       'existing_loan_amt', 'StartLoanApply', 'CompleteIDCertification',
       'EndLoanApply', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditinfo',
       'UseLoanManage', 'cnt', 'company_year', 'age_group', 'day',
       'credit_score_cat', 'bank_id_cat', 'product_id_cat',
       'loan_amt_per_once', 'debt_ratio', 'over_debt', 'da_per_income', 'pai',
       'total_income'],
      dtype='object')

In [23]:
X_smote.columns

Index(['loan_limit', 'loan_rate', 'gender', 'credit_score', 'yearly_income',
       'income_type', 'employment_type', 'houseown_type', 'desired_amount',
       'purpose', 'personal_rehabilitation_yn', 'existing_loan_cnt',
       'existing_loan_amt', 'StartLoanApply', 'CompleteIDCertification',
       'EndLoanApply', 'UsePrepayCalc', 'UseDSRCalc', 'GetCreditinfo',
       'UseLoanManage', 'cnt', 'company_year', 'age_group', 'day',
       'credit_score_cat', 'bank_id_cat', 'product_id_cat',
       'loan_amt_per_once', 'debt_ratio', 'over_debt', 'da_per_income', 'pai',
       'total_income'],
      dtype='object')

['EndLoanApply',
 'income_type',
 'purpose',
 'loan_limit',
 'day',
 'age_group',
 'credit_score',
 'UseLoanManage',
 'loan_rate',
 'gender',
 'existing_loan_cnt',
 'product_id_cat',
 'StartLoanApply',
 'desired_amount',
 'cnt',
 'credit_score_cat',
 'CompleteIDCertification']

In [34]:
# feature selection by importance of lgbm model
split_im_var = ["loan_rate", "credit_score", "EndLoanApply", "StartLoanApply", "age_group",
               "existing_loan_cnt", "desired_amount", "CompleteIDCertification",
               "UseLoanManage", "loan_limit", "income_type", "credit_score_cat",
               "cnt", "purpose", "gender", "product_id_cat", "yearly_income", "existing_laon_amt",
               "total_income", "day"]
gain_im_var = ["EndLoanApply", "StartLoanApply", "credit_score", "CompleteIDCertification", "loan_rate",
              "age_group", "UseLoanManage", "gender", "desired_amount", "existing_loan_cnt",
              "purpose", "income_type", "product_id_cat", "credit_score_cat", "cnt", "loan_limit",
              "GetCreditinfo", "day", "houseown_type", "da_per_income"]
shap_im_var = ["credit_score", "EndLoanApply", "StartLoanApply", "age_group", "loan_rate",
              "credit_score_cat", "CompleteIDCertification", "UseLoanManage", "existing_loan_cnt",
              "CompleteIDCertification", "UseLoanManage", "existing_loan_cnt", "purpose",
              "desired_amount", "income_type", "gender", "product_id_cat", "loan_limt", "day",
              "house_type", "cnt", "yearly_income", "total_income"]

split_gain_im_var = list(set(split_im_var)&set(gain_im_var))
sg_shap_im_var = list(set(split_gain_im_var)&set(shap_im_var))

print(len(split_im_var))
print(len(gain_im_var))
print(len(shap_im_var))
print(len(split_gain_im_var)) # split 과 gain 공통
print(len(sg_shap_im_var))

20
20
23
17
16


In [35]:
X_smote = X_smote[sg_shap_im_var]
# X_train = X_train[important_var]
X_val = X_val[sg_shap_im_var]
X_test = X_test[sg_shap_im_var]

In [ ]:
# n_estimators.. 400 기준은 없음
clf = LGBMClassifier(**best_params)

evals = [(X_val, y_val)]
clf.fit(X_smote, y_smote, early_stopping_rounds = 100, eval_metric = "auc",
       eval_set = evals, verbose = True)

pred_proba = clf.predict_proba(X_val)[:, 1]
preds = clf.predict(X_val)

[1]	valid_0's auc: 0.720521	valid_0's binary_logloss: 0.465878
[2]	valid_0's auc: 0.771989	valid_0's binary_logloss: 0.361284
[3]	valid_0's auc: 0.810123	valid_0's binary_logloss: 0.30517
[4]	valid_0's auc: 0.817845	valid_0's binary_logloss: 0.260984
[5]	valid_0's auc: 0.821268	valid_0's binary_logloss: 0.240229
[6]	valid_0's auc: 0.823128	valid_0's binary_logloss: 0.224081
[7]	valid_0's auc: 0.831784	valid_0's binary_logloss: 0.210284
[8]	valid_0's auc: 0.830232	valid_0's binary_logloss: 0.216606
[9]	valid_0's auc: 0.834259	valid_0's binary_logloss: 0.206132
[10]	valid_0's auc: 0.839267	valid_0's binary_logloss: 0.196539
[11]	valid_0's auc: 0.840456	valid_0's binary_logloss: 0.192112
[12]	valid_0's auc: 0.840315	valid_0's binary_logloss: 0.193466
[13]	valid_0's auc: 0.842001	valid_0's binary_logloss: 0.188275
[14]	valid_0's auc: 0.842575	valid_0's binary_logloss: 0.185
[15]	valid_0's auc: 0.843443	valid_0's binary_logloss: 0.182554
[16]	valid_0's auc: 0.843606	valid_0's binary_logloss

[130]	valid_0's auc: 0.862039	valid_0's binary_logloss: 0.166624
[131]	valid_0's auc: 0.862012	valid_0's binary_logloss: 0.166751
[132]	valid_0's auc: 0.862232	valid_0's binary_logloss: 0.16656
[133]	valid_0's auc: 0.862254	valid_0's binary_logloss: 0.166452
[134]	valid_0's auc: 0.862308	valid_0's binary_logloss: 0.166752
[135]	valid_0's auc: 0.862407	valid_0's binary_logloss: 0.166494
[136]	valid_0's auc: 0.862494	valid_0's binary_logloss: 0.167015
[137]	valid_0's auc: 0.862488	valid_0's binary_logloss: 0.167235
[138]	valid_0's auc: 0.862461	valid_0's binary_logloss: 0.166813
[139]	valid_0's auc: 0.862589	valid_0's binary_logloss: 0.166514
[140]	valid_0's auc: 0.862583	valid_0's binary_logloss: 0.166784
[141]	valid_0's auc: 0.86256	valid_0's binary_logloss: 0.16692
[142]	valid_0's auc: 0.862562	valid_0's binary_logloss: 0.167052
[143]	valid_0's auc: 0.862635	valid_0's binary_logloss: 0.166639
[144]	valid_0's auc: 0.862641	valid_0's binary_logloss: 0.166765
[145]	valid_0's auc: 0.86263

[257]	valid_0's auc: 0.864929	valid_0's binary_logloss: 0.164164
[258]	valid_0's auc: 0.8649	valid_0's binary_logloss: 0.164152
[259]	valid_0's auc: 0.864896	valid_0's binary_logloss: 0.164217
[260]	valid_0's auc: 0.864888	valid_0's binary_logloss: 0.164275
[261]	valid_0's auc: 0.864881	valid_0's binary_logloss: 0.164369
[262]	valid_0's auc: 0.864992	valid_0's binary_logloss: 0.164253
[263]	valid_0's auc: 0.864992	valid_0's binary_logloss: 0.164198
[264]	valid_0's auc: 0.86503	valid_0's binary_logloss: 0.16415
[265]	valid_0's auc: 0.865068	valid_0's binary_logloss: 0.164101
[266]	valid_0's auc: 0.86512	valid_0's binary_logloss: 0.164063
[267]	valid_0's auc: 0.865129	valid_0's binary_logloss: 0.164119
[268]	valid_0's auc: 0.865154	valid_0's binary_logloss: 0.164075
[269]	valid_0's auc: 0.865148	valid_0's binary_logloss: 0.164185
[270]	valid_0's auc: 0.865167	valid_0's binary_logloss: 0.164104
[271]	valid_0's auc: 0.865162	valid_0's binary_logloss: 0.164151
[272]	valid_0's auc: 0.865207	

In [ ]:
def get_clf_eval(test, pred, pred_proba) :
    confusion = confusion_matrix(test, pred)
    accuracy = accuracy_score(test, pred)
    precision = precision_score(test, pred)
    recall = recall_score(test, pred)
    f1 = f1_score(test, pred)
    roc_auc = roc_auc_score(test, pred)
    fpr, tpr, thresholds = roc_curve(test, pred_proba, pos_label = 1)
    
    print("오차행렬")
    print(confusion)
    print(f"정확도 : {accuracy:.4f}, 정밀도 : {precision:.4f}, 재현율 : {recall:.4f}, F1 : {f1:.4f}, AUC : {roc_auc:.4f}")
    return confusion, accuracy, precision, recall, f1, fpr, tpr, thresholds
    
    
confusion, accuracy, precision, recall, f1, fpr, tpr, thresholds = get_clf_eval(y_val, preds, pred_proba)

## roc_auc curve

In [ ]:
def roc_auc_curve_plot(fpr, tpr, fpr_train = None, tpr_train = None, fpr_test=None, tpr_test=None) :
    plt.figure(figsize = (8, 8))
    plt.plot([0,1], [0,1], label = "Random")
    plt.plot(fpr, tpr, label = "ROC_valid")
    
    if fpr_train is not None : 
        plt.plot(fpr_train, tpr_train, label = "ROC_train")
    
    if fpr_test is not None : 
        plt.plot(fpr_test, tpr_test, label = "ROC_test")

    plt.xlabel("FPR")
    plt.ylabel("TPR")

    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# train_set
pred_proba_train = clf.predict_proba(X_smote)[:, 1]
preds_train = clf.predict(X_smote)

# test set
pred_proba_test = clf.predict_proba(X_test)[:, 1]
preds_test = clf.predict(X_test)

# result
result_train = list(get_clf_eval(y_smote, preds_train, pred_proba_train))
result = list(get_clf_eval(y_test, preds_test, pred_proba_test))

In [ ]:
# result
result_train = list(get_clf_eval(y_smote, preds_train, pred_proba_train))
result = list(get_clf_eval(y_test, preds_test, pred_proba_test))

In [ ]:
# Val set , Test set ROC_AUC graphh
roc_auc_curve_plot(fpr, tpr, result_train[5], result_train[6],result[5], result[6])

# Model Save

In [ ]:
# save model 
save_num = 1

# joblib.dump(clf, f"..\\..\\model\\lgb_{save_num}.pkl")

# load_model 
load_model = joblib.load(f"..\\..\\model\\lgb_{save_num}.pkl")

In [ ]:
# temp
preds_train = load_model.predict(X_smote)

# Make DBSCAN clustering data (for downsampling)
- 잘 예측하지 못한 데이터는 줄여주기

# Make DBSMOTE data (for upsampling) 
- 잘 예측한 것들은 늘려주기

In [ ]:
# y_smote 와 predicts
cluster_df = y_smote.copy()
cluster_df["y_pred"] = preds_train

cluster_df["score"] = cluster_df["y_pred"] + cluster_df["is_applied"]

true_index = np.where(cluster_df["score"] != 2.)
false_index = np.where(cluster_df["score"] == 2.)

# X_db, y_db : DBSCAN을 하기위한 데이터 셋 (prediction 이 false)
X_db = X_smote.iloc[false_index[0]]
y_db = y_smote.iloc[false_index[0]]

X_db_sm = X_smote.iloc[true_index[0]]
y_db_sm = y_smote.iloc[true_index[0]]

# one_hot_var = ["age_group", "gender", "income_type", "purpose", "day", "product_id_cat", "employment_type",
#                "bank_id_cat", "houseown_type", "company_year", "credit_score_cat"]

# from sklearn.preprocessing import OneHotEncoder
# # one_hot encoding dataframe
# onehot_encoder = OneHotEncoder()
# onehot = onehot_encoder.fit_transform(X_db[one_hot_var])

# onehot = onehot.toarray()
# onehot = pd.DataFrame(onehot)
# onehot.columns = onehot_encoder.get_features_names()

# min_max_var = list(set(X_db.columns) - set(one_hot_var))
min_max_var = ["credit_score", "loan_rate", "desired_amount", "existing_loan_cnt", "loan_limit", "yearly_income",
              "existing_loan_amt", "total_income"]

minmax_scaler = MinMaxScaler()
minmax = minmax_scaler.fit_transform(X_db[min_max_var])

# numerical_data (minmax)
minmax = pd.DataFrame(minmax, columns = min_max_var)

# concatenate
# encoding_df = pd.concat([onehot, minmax], axis = 1)

# from sklearn.cluster import DBSCAN
# dummy (one_hot encoding) , min_max_scaling

# dbscan
# dbscan = DBSCAN(eps = 0.6, min_samples = 8, metric = "euclidean")
# dbscan_labels = dbscan.fit_predict(minmax)

# kmeans_clustering n_cluster = 20 (temp)
kmeans = KMeans(n_clusters = 20, random_state = 0).fit(minmax)

# cluster label 추가
# y_smote 에서 index 를 안 좋은 cluster 에 들어있는 행들 index 를 뽑아 내서 
# X_smote 에 적용 시켜야 함
y_db["cluster"] = kmeans.labels_
y_db["pred"] = cluster_df.iloc[false_index, 1] # 

In [ ]:
# cluster 확인
print(y_db.index)

y_db.groupby("cluster").sum()

y_db[y_db["cluster"] == ...].index

In [ ]:
# stop

In [ ]:
import lightgbm as lgb
from imblearn.over_sampling import *

smote_ = SMOTE(random_state = 0)

# fitting
X_db_sm, y_db_sm = smote_.fit_resample(X_db_sm, y_db_sm)

# 다시 학습시킬 데이터셋 만들기 + 저장 ?
X_train_new = pd.concat([X_db, X_db_sm], axis = 0).reset_index()
y_train_new = pd.concat([y_db, y_db_sm], axis = 0).reset_index()

# 10번 반복

In [ ]:
import math
import time
import datetime
import tqdm

pred_proba_ls = []
preds_ls = []
X_df_ls = [X_smote]
y_df_ls = [y_smote]

def greedy_down_up_sampling(X, y,save_num)
    clf = LGBMClassifier(**best_params)

    evals = [(X_val, y_val)]
    clf.fit(X,, y, early_stopping_rounds = 100, eval_metric = "auc",
           eval_set = evals, verbose = True)

    pred_proba = clf.predict_proba(X_val)[:, 1]
    preds = clf.predict(X_val)
    
    pred_proba_ls.append(pred_proba)
    preds_ls.append(preds)
    
    get_clf_eval(y_val, preds, pred_proba)
    
    joblib.dump(clf, f"..\\..\\model\\lgb_{save_num}.pkl")

#     # load_model 
#     load_model = joblib.load(f"..\\..\\model\\lgb_{save_num}.pkl")
    
    # y_smote 와 predicts
    dbscan_df = y.copy()
    dbscan_df["y_pred"] = preds

    dbscan_df["score"] = dbscan_df["y_pred"] + dbscan_df["is_applied"]

    true_index = np.where(dbscan_df["score"] != 2.)
    false_index = np.where(dbscan_df["score"] == 2.)

    # X_db, y_db : DBSCAN을 하기위한 데이터 셋 (prediction 이 false)
    X_db = X.iloc[false_index[0]]
    y_db = y.iloc[false_index[0]]

    X_db_sm = X.iloc[true_index[0]]
    y_db_sm = y.iloc[true_index[0]]
    
#     one_hot_var = ["age_group", "gender", "income_type", "purpose", "day", "product_id_cat", "employment_type",
#                "bank_id_cat", "houseown_type", "company_year", "credit_score_cat"]

#     from sklearn.preprocessing import OneHotEncoder
#     # one_hot encoding dataframe
#     onehot_encoder = OneHotEncoder()
#     onehot = onehot_encoder.fit_transform(X_db[one_hot_var])

#     onehot = onehot.toarray()
#     onehot = pd.DataFrame(onehot)
#     onehot.columns = onehot_encoder.get_features_names()

    # min_max_var = list(set(X_db.columns) - set(one_hot_var))
    min_max_var = ["credit_score", "loan_rate", "desired_amount", "existing_loan_cnt", "loan_limit", "yearly_income",
                  "existing_loan_amt", "total_income"]

    

    minmax_scaler = MinMaxScaler()
    minmax = minmax_scaler.fit_transform(X_db[min_max_var])

    # numerical_data
    minmax = pd.DataFrame(minmax, columns = min_max_var)
    
#     # dbscan
#     dbscan = DBSCAN(eps = 0.6, min_samples = 100, metric = "euclidean")
#     dbscan_labels = dbscan.fit_predict(minmax)

    # cluster label 추가
    y_db["dbscan_cluster"] = dbscan_labels
    
    smote_ = SMOTE(random_state = 0)

    # fitting
    X_db_sm, y_db_sm = smote_.fit_resample(X_db_sm, y_db_sm)

    # 다시 학습시킬 데이터셋 만들기 + 저장 ?
    X_train_new = pd.concat([X_db, X_db_sm], axis = 0).reset_index()
    y_train_new = pd.concat([y_db, y_db_sm], axis = 0).reset_index()
    
    X_df_ls.append(X_train_new)
    y_df_ls.append(y_train_new)
    
    return X_train_new, y_train_new

start = time.time()

for i in tqdm(range(10)) : 
    X, y = X_df_ls[i], y_df_ls[i]
    greedy_down_up_sampling(X, y, i)

# 총 걸리는 시간 측정
end = time.time()
sec = (end - start)

result_time = datetime.timedelta(seconds=sec)
print(result_time)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])